# [SciPy](https://docs.scipy.org/doc/scipy-1.15.0/index.html#)

<img src="../../Auxiliary_Files/Graphics/Data_Structures/scipy_logo.png" alt="SciPy_Logo" width="600" height="300">

## Table of Contents
- [Introduction](#introduction)
- [Overview](#overview)
- [Arrays](#arrays)
    - [Basic Properties](#basic-properites)
    - [Array Creation Methods]()
    - [Example: Analyzing a 2D Image](#example-analyzing-a-2d-image)
    - [Handling NANS](#handling-nans)
    - [Stacking and Exporting Arrays](#stacking-and-exporting-arrays)
    - [Sorting & Searching](#sorting--searching)
- [Mathematical Routines](#mathematical-routines)
    - [General Functions](#basic-mathematical-functions)
    - [Statistics](#statistics)
    - [Linear Algebra](#linear-algebra)
    - [Simple Polynomial Regression Methods](#simple-polynomial-regression-methods)
- [Exercises](#exercises)
    - [Problem 1: Creating an Array in Four Different Ways](#problem-1-creating-an-array-in-four-different-ways)
    - [Problem 2: Handling NANs](#problem-2-handling-nans)
    - [Problem 3: Modeling Ingress and Egress](#problem-3-modeling-ingress-and-egress)

## Introduction

The [SciPy](https://docs.scipy.org/doc/scipy-1.15.0/index.html#) package contains a slew of different analysis methods and tools that are relevant in exoplanet research. This lesson will focus on these methods, and outline several use cases. 

 <div class="alert alert-block alert-warning">

**NOTE**: SciPy has many high level functions and optimization routines that are largely out of scope of ExoCore. These include routines that may be relevant in other aspects of astrophysics research; if you are interested, check out all SciPy modules [here](https://docs.scipy.org/doc/scipy-1.15.0/reference/index.html).

Before we begin, run the code block below to activate the interactive portions of this lesson:

In [2]:
import scipy
from jupyterquiz import display_quiz
import json
with open("../../Exercise_Solutions/Module_3/SciPy/Checkpoints/questions.json", "r") as file:
    questions=json.load(file)

## Constants Module

The first module in SciPy is the Constants module. The utility is in the name; this contains many relevant constants relevant in science and mathematics.

In [7]:
## Evoke pi using scipy.constants.pi

print("The value of pi is: " + str(scipy.constants.pi))

## Speed of light...

c = scipy.constants.c
m_n = scipy.constants.m_n
print("The speed of light is: " + str(c))
print("The mass of the neutron is: " + str(m_n))

The value of pi is: 3.141592653589793
The speed of light is: 299792458.0
The mass of the neutron is: 1.67492749804e-27


You can find a list of available units [here](https://docs.scipy.org/doc/scipy-1.15.0/reference/constants.html). All units are reported in [SI units](https://en.wikipedia.org/wiki/International_System_of_Units#:~:text=The%20SI%20comprises%20a%20coherent,candela%20(cd%2C%20luminous%20intensity)).